# Consolidación de los conjuntos de datos

Por medio del siguiente notebook se va a consolidar la estructura y las columnas a analizar durante el trabajo, el proceso a realizar será el siguiente:
- Combinación de los conjuntos de datos relacionados con la información de la persona cómo el hogar, vivienda.
- Adición de la columna `P6240` que hace referencia a la pregunta, ¿Cuál fue la actividad que realizó la semana pasada? En materia de Fuerza de trabajo, el objetivo del trabajo es el de realizar un modelo de regresión logística sobre esta variable, en las respuestas 1 y 2, que se refieren a los casos donde la persona se encuentra trabajando o está buscando trabajo respectivamente.
- Se eliminarán además, aquellas columnas identificadoras que no son relevantes para el trabajo

In [1]:
# Carga de las librerias necesarias para la consolidación
import pandas as pd
import os

In [2]:
'''
Diccionario que contiene los nombres con los que se van a identificar las distintas variables codíficadas de la GEIH según su pregunta asociada.
'''
rename_cols = {
    'AREA': 'AREA_METROPOLITANA',
    'CLASE': 'ZONA_TERRITORIAL',
    'FEX_C18': 'FACTOR_EXPANSION',
    'P3271': 'SEXO',
    'P6040': 'EDAD',
    'P6030S1': 'MES_NACIMIENTO',
    'P6030S3': 'ANIO_NACIMIENTO',
    'P6050': 'PARENTESCO_JEFE_DE_HOGAR',
    'P6083': 'MADRE_RESIDE_HOGAR',
    'P6081': 'PADRE_RESIDE_HOGAR',
    'P2057': 'SE_CONSIDERA_CAMPESINO',
    'P2059': 'ALGUNA_VEZ_CAMPESINO',
    'P2061': 'COMUNIDAD_ES_CAMPESINA',
    'P6080': 'ETNIA',
    'P6070': 'ESTADO_CIVIL',
    'P6071': 'CONYUGE_VIVE_HOGAR',
    'P6090': 'AFILIADO_SALUD',
    'P6100': 'TIPO_REGIMEN_SALUD',
    'P6110': 'QUIEN_PAGA_SALUD',
    'P6120': 'VALOR_SALUD',
    'P1906S1': 'DIFICULTAD_OIR',
    'P1906S2': 'DIFICULTAD_HABLAR',
    'P1906S3': 'DIFICULTAD_VER',
    'P1906S4': 'DIFICULTAD_MOVER_SUBIR_BAJAR',
    'P1906S5': 'DIFICULTAD_AGARRAR',
    'P1906S6': 'DIFICULTAD_ENTENDER',
    'P1906S7': 'DIFICULTAD_BANARSE_VESTIRSE',
    'P1906S8': 'DIFICULTAD_RELACIONARSE',
    'P6160': 'LEER_ESCRIBIR',
    'P6170': 'ACTUALMENTE_ESTUDIA',
    'P3041': 'INSTITUCION_PUBLICA_PRIVADA',
    'P3042': 'MAYOR_NIVEL_EDUCATIVO',
    'P3042S1': 'MAYOR_GRADO_ALCANZADO',
    'P3043': 'TITULO_OBTENIDO',
    'P3038': 'ORIENTACION_SEXUAL',
    'P3039': 'GENERO',
    'P6240': 'ACTIVIDAD_OCUPADA_ULTIMA_SEMANA',
    'P4000': 'TIPO_VIVIENDA',
    'P4010': 'MATERIAL_PAREDES',
    'P4020': 'MATERIAL_PISOS',
    'P4030S1': 'SERVICIOS_ENERGIA_ELECTRICA',
    'P4030S1A1': 'ESTRATO_ENERGIA_ELECTRICA',
    'P4030S2': 'SERVICIOS_GAS_NATURAL',
    'P4030S3': 'SERVICIOS_ALCANTARILLADO',
    'P4030S4': 'SERVICIOS_RECOLECCION_BASURAS',
    'P4030S5': 'SERVICIOS_ACUEDUCTO',
    'P70': 'NUMERO_HOGARES_VIVIENDA',
    'P5000': 'NUMERO_CUARTOS_VIVIENDA',
    'P5010': 'NUMERO_CUARTOS_DORMIR',
    'P5020': 'TIPO_SANITARIO',
    'P5030': 'USO_COMPARTIDO_SANITARIO',
    'P5040': 'COMO_ELIMINA_BASURA',
    'P5050': 'DONDE_OBTIENE_AGUA',
    'P5070': 'DONDE_PREPARA_ALIMENTOS',
    'P5080': 'TIPO_COMBUSTIBLE_COCINAR',
    'P5090': 'TIPO_OCUPACION_VIVIENDA',
    'P5222S1': 'HOGAR_TIENE_CUENTA_CORRIENTE',
    'P5222S2': 'HOGAR_TIENE_CUENTA_AHORROS',
    'P5222S3': 'HOGAR_TIENE_CDT',
    'P5222S4': 'HOGAR_TIENE_PRESTAMO_COMPRA_VIVIENDA',
    'P5222S5': 'HOGAR_TIENE_PRESTAMO_COMPRA_VEHICULO',
    'P5222S6': 'HOGAR_TIENE_PRESTAMO_LIBRE_INVERSION',
    'P5222S7': 'HOGAR_TIENE_TARJETA_CREDITO',
    'P5222S9': 'HOGAR_TIENE_NINGUNO_DE_ESTOS',
    'P5222S10':'HOGAR_NO_SABE_FINANCIERO',
    'P5222S8':'HOGAR_TIENE_OTRO_FINANCIERO',
    'P6008': 'NUMERO_PERSONAS_HOGAR',
}

In [3]:
'''
Listado de Columnas identificadoras o cuya información no es relevante para el objetivo del trabajo
'''
drop_columns = [
    'PERIODO',
    'PER',
    'HOGAR',
    'REGIS',
    'PT',
    'POB_MAY18',
    'P6016',
    'P6083S1',
    'P6081S1',
    'P6080S1',
    'P6080S1A1',
    'P6071S1',
    'P6071S1',
    'P3147S1',
    'P3147S2',
    'P3147S3',
    'P3147S4',
    'P3147S5',
    'P3147S6',
    'P3147S7',
    'P3147S8',
    'P3147S9',
    'P3147S10',
    'P3147S11',
    'P3147S10A1',
    'P3042S2',
    'P3043S1',
    'P4030S4A1',
    'P5090S1',
    'P5100',
    'P5110',
    'P5130',
    'P5140',
    'P5222S8A1'
]

In [4]:
'''
Se define la función que permite realizar la consolidación de los conjuntos de datos.
'''
def consolidate_and_clean_data(personas:pd.DataFrame, fuerza_trabajo:pd.DataFrame, hogares:pd.DataFrame)->pd.DataFrame:

    info_trabajo = fuerza_trabajo[['DIRECTORIO', 'SECUENCIA_P', 'ORDEN','P6240']]
    result = personas.merge(info_trabajo, on= ['DIRECTORIO', 'SECUENCIA_P', 'ORDEN'], how='left')
    
    interseccion= list(set(hogares.columns).intersection(set(result.columns)))
    interseccion_del= [col for col in interseccion if col not in ['DIRECTORIO', 'SECUENCIA_P', 'ORDEN', 'HOGAR']]
    hogares = hogares.drop(columns= interseccion_del)
    result_2 = result.merge(hogares, on= ['DIRECTORIO', 'SECUENCIA_P', 'HOGAR'], how='left')
    
    final = result_2.drop(columns= drop_columns)
    final_renamed = final.rename(columns=rename_cols)
    
    extra_cols_to_drop = ['MES_NACIMIENTO','ANIO_NACIMIENTO']
    final_renamed = final_renamed.drop(columns=extra_cols_to_drop)
    
    more_columns = ['LGB_Numerica','LGB_sectores','LGBT_Numerica','Trans_numerica','Discapacidad','Dificultad','Campesina']
    
    try:
        final_renamed = final_renamed.drop(columns=more_columns)
    except:
        pass

    return final_renamed

In [5]:
'''
Cargue de los conjuntos de datos Originales
'''

#Enero
enero_personas = pd.read_csv(r"C:\Users\andre\OneDrive\Documentos\Maestría\Proyecto de Grado\Datos\01_Enero\CSV\Características generales, seguridad social en salud y educación.CSV" ,
                             encoding='latin-1', sep=';')
enero_trabajo = pd.read_csv(r"C:\Users\andre\OneDrive\Documentos\Maestría\Proyecto de Grado\Datos\01_Enero\CSV\Fuerza de trabajo.CSV" ,
                            encoding='latin-1', sep=';')
enero_hogares = pd.read_csv(r"C:\Users\andre\OneDrive\Documentos\Maestría\Proyecto de Grado\Datos\01_Enero\CSV\Datos del hogar y la vivienda.CSV" ,
                            encoding='latin-1', sep=';')

# Febrero
febrero_personas = pd.read_csv(r"C:\Users\andre\OneDrive\Documentos\Maestría\Proyecto de Grado\Datos\02_Febrero\CSV\Características generales, seguridad social en salud y educación.CSV" ,
                               encoding='latin-1', sep=';')
febrero_trabajo = pd.read_csv(r"C:\Users\andre\OneDrive\Documentos\Maestría\Proyecto de Grado\Datos\02_Febrero\CSV\Fuerza de trabajo.CSV" ,
                              encoding='latin-1', sep=';')
febrero_hogares = pd.read_csv(r"C:\Users\andre\OneDrive\Documentos\Maestría\Proyecto de Grado\Datos\02_Febrero\CSV\Datos del hogar y la vivienda.CSV" ,
                              encoding='latin-1', sep=';')

# Marzo
marzo_personas = pd.read_csv(r"C:\Users\andre\OneDrive\Documentos\Maestría\Proyecto de Grado\Datos\03_Marzo\CSV\Características generales, seguridad social en salud y educación.CSV" ,
                             encoding='latin-1', sep=';')
marzo_trabajo = pd.read_csv(r"C:\Users\andre\OneDrive\Documentos\Maestría\Proyecto de Grado\Datos\03_Marzo\CSV\Fuerza de trabajo.CSV" ,
                            encoding='latin-1', sep=';')
marzo_hogares = pd.read_csv(r"C:\Users\andre\OneDrive\Documentos\Maestría\Proyecto de Grado\Datos\03_Marzo\CSV\Datos del hogar y la vivienda.CSV" ,
                            encoding='latin-1', sep=';')

# Abril
abril_personas = pd.read_csv(r"C:\Users\andre\OneDrive\Documentos\Maestría\Proyecto de Grado\Datos\04_Abril\CSV\Características generales, seguridad social en salud y educación.CSV" ,
                             encoding='latin-1', sep=';')
abril_trabajo = pd.read_csv(r"C:\Users\andre\OneDrive\Documentos\Maestría\Proyecto de Grado\Datos\04_Abril\CSV\Fuerza de trabajo.CSV" ,
                            encoding='latin-1', sep=';')
abril_hogares = pd.read_csv(r"C:\Users\andre\OneDrive\Documentos\Maestría\Proyecto de Grado\Datos\04_Abril\CSV\Datos del hogar y la vivienda.CSV" ,
                            encoding='latin-1', sep=';')

# Mayo
mayo_personas = pd.read_csv(r"C:\Users\andre\OneDrive\Documentos\Maestría\Proyecto de Grado\Datos\05_Mayo\CSV\Características generales, seguridad social en salud y educación.CSV" ,
                            encoding='latin-1', sep=';')
mayo_trabajo = pd.read_csv(r"C:\Users\andre\OneDrive\Documentos\Maestría\Proyecto de Grado\Datos\05_Mayo\CSV\Fuerza de trabajo.CSV" ,
                           encoding='latin-1', sep=';')
mayo_hogares = pd.read_csv(r"C:\Users\andre\OneDrive\Documentos\Maestría\Proyecto de Grado\Datos\05_Mayo\CSV\Datos del hogar y la vivienda.CSV" ,
                           encoding='latin-1', sep=';')

# Junio
junio_personas = pd.read_csv(r"C:\Users\andre\OneDrive\Documentos\Maestría\Proyecto de Grado\Datos\06_Junio\CSV\Características generales, seguridad social en salud y educación.CSV" ,
                             encoding='latin-1', sep=';')
junio_trabajo = pd.read_csv(r"C:\Users\andre\OneDrive\Documentos\Maestría\Proyecto de Grado\Datos\06_Junio\CSV\Fuerza de trabajo.CSV" ,
                            encoding='latin-1', sep=';')
junio_hogares = pd.read_csv(r"C:\Users\andre\OneDrive\Documentos\Maestría\Proyecto de Grado\Datos\06_Junio\CSV\Datos del hogar y la vivienda.CSV" ,
                            encoding='latin-1', sep=';')

# Julio
julio_personas = pd.read_csv(r"C:\Users\andre\OneDrive\Documentos\Maestría\Proyecto de Grado\Datos\07_Julio\CSV\Características generales, seguridad social en salud y educación.CSV" ,
                             encoding='latin-1', sep=';')
julio_trabajo = pd.read_csv(r"C:\Users\andre\OneDrive\Documentos\Maestría\Proyecto de Grado\Datos\07_Julio\CSV\Fuerza de trabajo.CSV" ,
                            encoding='latin-1', sep=';')
julio_hogares = pd.read_csv(r"C:\Users\andre\OneDrive\Documentos\Maestría\Proyecto de Grado\Datos\07_Julio\CSV\Datos del hogar y la vivienda.CSV" ,
                            encoding='latin-1', sep=';')

# Agosto
agosto_personas = pd.read_csv(r"C:\Users\andre\OneDrive\Documentos\Maestría\Proyecto de Grado\Datos\08_Agosto\CSV\Características generales, seguridad social en salud y educación.CSV" ,
                              encoding='latin-1', sep=';')
agosto_trabajo = pd.read_csv(r"C:\Users\andre\OneDrive\Documentos\Maestría\Proyecto de Grado\Datos\08_Agosto\CSV\Fuerza de trabajo.CSV" ,
                             encoding='latin-1', sep=';')
agosto_hogares = pd.read_csv(r"C:\Users\andre\OneDrive\Documentos\Maestría\Proyecto de Grado\Datos\08_Agosto\CSV\Datos del hogar y la vivienda.CSV" ,
                             encoding='latin-1', sep=';')

# Septiembre
septiembre_personas = pd.read_csv(r"C:\Users\andre\OneDrive\Documentos\Maestría\Proyecto de Grado\Datos\09_Septiembre\CSV\Características generales, seguridad social en salud y educación.CSV" ,
                                  encoding='latin-1', sep=';')
septiembre_trabajo = pd.read_csv(r"C:\Users\andre\OneDrive\Documentos\Maestría\Proyecto de Grado\Datos\09_Septiembre\CSV\Fuerza de trabajo.CSV" ,
                                 encoding='latin-1', sep=';')
septiembre_hogares = pd.read_csv(r"C:\Users\andre\OneDrive\Documentos\Maestría\Proyecto de Grado\Datos\09_Septiembre\CSV\Datos del hogar y la vivienda.CSV",
                                 encoding='latin-1', sep=';')

# Octubre
octubre_personas = pd.read_csv(r"C:\Users\andre\OneDrive\Documentos\Maestría\Proyecto de Grado\Datos\10_Octubre\CSV\Características generales, seguridad social en salud y educación.CSV",
                                  encoding='latin-1', sep=';')
octubre_trabajo = pd.read_csv(r"C:\Users\andre\OneDrive\Documentos\Maestría\Proyecto de Grado\Datos\10_Octubre\CSV\Fuerza de trabajo.CSV",
                                 encoding='latin-1', sep=';')
octubre_hogares = pd.read_csv(r"C:\Users\andre\OneDrive\Documentos\Maestría\Proyecto de Grado\Datos\10_Octubre\CSV\Datos del hogar y la vivienda.CSV",
                                 encoding='latin-1', sep=';')

# Noviembre
noviembre_personas = pd.read_csv(r"C:\Users\andre\OneDrive\Documentos\Maestría\Proyecto de Grado\Datos\11_Noviembre\CSV\Características generales, seguridad social en salud y educación.CSV",
                                  encoding='latin-1', sep=';')
noviembre_trabajo = pd.read_csv(r"C:\Users\andre\OneDrive\Documentos\Maestría\Proyecto de Grado\Datos\11_Noviembre\CSV\Fuerza de trabajo.CSV",
                                 encoding='latin-1', sep=';')
noviembre_hogares = pd.read_csv(r"C:\Users\andre\OneDrive\Documentos\Maestría\Proyecto de Grado\Datos\11_Noviembre\CSV\Datos del hogar y la vivienda.CSV",
                                 encoding='latin-1', sep=';')

C:\Users\andre\AppData\Local\Temp\ipykernel_21328\3597765941.py:8: DtypeWarning: Columns (34) have mixed types. Specify dtype option on import or set low_memory=False.
  enero_trabajo = pd.read_csv(r"C:\Users\andre\OneDrive\Documentos\Maestría\Proyecto de Grado\Datos\01_Enero\CSV\Fuerza de trabajo.CSV" ,
C:\Users\andre\AppData\Local\Temp\ipykernel_21328\3597765941.py:14: DtypeWarning: Columns (43) have mixed types. Specify dtype option on import or set low_memory=False.
  febrero_personas = pd.read_csv(r"C:\Users\andre\OneDrive\Documentos\Maestría\Proyecto de Grado\Datos\02_Febrero\CSV\Características generales, seguridad social en salud y educación.CSV" ,
C:\Users\andre\AppData\Local\Temp\ipykernel_21328\3597765941.py:16: DtypeWarning: Columns (34) have mixed types. Specify dtype option on import or set low_memory=False.
  febrero_trabajo = pd.read_csv(r"C:\Users\andre\OneDrive\Documentos\Maestría\Proyecto de Grado\Datos\02_Febrero\CSV\Fuerza de trabajo.CSV" ,
C:\Users\andre\AppData\L

In [6]:
# Ejecución de la función
enero = consolidate_and_clean_data(enero_personas, enero_trabajo, enero_hogares)
febrero = consolidate_and_clean_data(febrero_personas, febrero_trabajo, febrero_hogares)
marzo = consolidate_and_clean_data(marzo_personas, marzo_trabajo, marzo_hogares)
abril = consolidate_and_clean_data(abril_personas, abril_trabajo, abril_hogares)
mayo = consolidate_and_clean_data(mayo_personas, mayo_trabajo, mayo_hogares)
junio = consolidate_and_clean_data(junio_personas, junio_trabajo, junio_hogares)
julio = consolidate_and_clean_data(julio_personas, julio_trabajo, julio_hogares)
agosto = consolidate_and_clean_data(agosto_personas, agosto_trabajo, agosto_hogares)
septiembre = consolidate_and_clean_data(septiembre_personas, septiembre_trabajo, septiembre_hogares)
octubre = consolidate_and_clean_data(octubre_personas, octubre_trabajo, octubre_hogares)
noviembre = consolidate_and_clean_data(noviembre_personas, noviembre_trabajo, noviembre_hogares)

In [7]:
# Exportación de los conjuntos de datos realizados
datos = [enero, febrero, marzo, abril, mayo, junio, julio, agosto, septiembre, octubre, noviembre]
nombres = ['Enero','Febrero','Marzo','Abril','Mayo','Junio','Julio','Agosto','Septiembre','Octubre','Noviembre']
for tabla, nombre in zip(datos,nombres):
    path = r"C:\Users\andre\OneDrive\Escritorio\Proyecto de Grado\result\db_consolidado_mensual"
    ruta_folder = os.path.join(path, nombre + '.csv')
    tabla.to_csv(ruta_folder, index=False, sep = ';', encoding='latin-1')